# Intro to REST APIs

This notebook demonstrates how to create requests and parse responses for the Data and Orders API with a quick code snippet at the end dedicated to the subscription API. We will create a search request for the Data API using  a `Geojson`. We will parse the response for image IDs. We will use those image IDs to place a order request.


More reference information can be found at [Ordering & Delivery](https://developers.planet.com/apis/orders/).

In [ ]:
import json
import os
import pathlib
import time

import requests
from requests.auth import HTTPBasicAuth

## Authenticating
You can also just paste your API key instead of the ```os.getenv('PL_API_KEY')```

In [ ]:
# API Key stored as an env variable
PLANET_API_KEY = os.getenv('PL_API_KEY')

In [ ]:
orders_url = 'https://api.planet.com/compute/ops/orders/v2/'
data_url = "https://api.planet.com/data/v1"

In [ ]:
auth = HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(data_url, auth=auth)
response

## Searching with the Data API
We can use the [data API](https://developers.planet.com/docs/apis/data/) in order to automate searching based on the search criterias like: date range, cloud cover, area cover, aoi. We can create an AOI using https://geojson.io/

In [ ]:
def parse_geojson(filename):
    features = json.load(open(filename))
    if "type" in features and features["type"] == "FeatureCollection":
        for f in features["features"]:
            geoms =  f["geometry"]
    elif "type" in features and features["type"] == "Feature":
        geoms= features["geometry"]
    else:
        geoms = features

    return geoms

In [ ]:
geometry = parse_geojson("sf.geojson")
geometry

In [ ]:
# get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geometry
}

# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte":"2023-01-15T00:00:00Z",
    "lte":"2023-01-17T00:00:00Z"
  }
}

# only get images which have >50% clear pixels
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "clear_percent",
  "config": {
    "gt": 50
  }
}

# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [ ]:
item_type = "PSScene"

# API request object
search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}
search_request

In [ ]:

# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=search_request)

# extract image IDs only
image_ids = [feature['id'] for feature in search_result.json()['features']]
print(len(image_ids))

In [ ]:
search_result.json()['_links']

### Requests example

In this notebook, we will be using `requests` to communicate with the orders v2 API. First, we will check our orders list to make sure authentication and communication is working as expected.

We want to get a response code of `200` from this API call. To troubleshoot other response codes, see the [List Orders](https://developers.planet.com/apis/orders/reference/) AOI reference.

In [ ]:
auth = HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(orders_url, auth=auth)
response

## Ordering

In this example, we will order two `PSScene` analytic images. For variations on this kind of order, see [Ordering Data](https://developers.planet.com/apis/orders/scenes/).

In this order, we request an `analytic` bundle. A bundle is a group of assets for an item. The `analytic` bundle for the  `PSScene` item contains 3 assets: the analytic image, the analytic xml file, and the udm. See the [Product bundles reference](https://developers.planet.com/docs/orders/product-bundles-reference/) to learn about other bundles and other items.

Now we will list the names of orders we have created thus far. Your list may be empty if you have not created an order yet.

In [ ]:
orders = response.json()['orders']
[r['name'] for r in orders[:5]]

### Place Order

In [ ]:
# set content type to json
headers = {'content-type': 'application/json'}

In [ ]:
# define products part of order
single_product = [
    {
      "item_ids": ["20220628_183020_20_248c"],
      "item_type": "PSScene",
      "product_bundle": "analytic_udm2"
    }
]

same_src_products = [
    {
      "item_ids": ["20151119_025740_0c74",
                   "20151119_025739_0c74"],
      "item_type": "PSScene",
      "product_bundle": "analytic_udm2"
    }
]

multi_src_products = [
    {
      "item_ids": ["20151119_025740_0c74"],
      "item_type": "PSScene",
      "product_bundle": "analytic_udm2"
    },
    {
      "item_ids": ["20220628_183020_20_248c"],
      "item_type": "PSScene",
      "product_bundle": "visual"
    },
    
]

In [ ]:
product = [
    {
      "item_ids": [image_ids[0]],
      "item_type": "PSScene",
      "product_bundle": "v"
    }
]


request = {  
   "name":"San Francisco",
   "products": product,
    "delivery": {"single_archive": True, "archive_type": "zip"}
}

In [ ]:
def place_order(request, auth):
    response = requests.post(orders_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response.json())
    order_id = response.json()['id']
    print(order_id)
    order_url = orders_url + '/' + order_id
    return order_url

In [ ]:
order_url = place_order(request, auth)

### Poll for Order Success

In [ ]:
def poll_for_success(order_url, auth, num_loops=30):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=auth)
        response = r.json()
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(30)
        
poll_for_success(order_url, auth)

In [ ]:
# define the clip tool
clip = {
    "clip": {
        "aoi": geometry
    }
}

In [ ]:
bandmath = {
  "bandmath": {
    "b1": "b1",
    "b2": "b2",
    "b3": "b3",
    "b4": "b4",
    "b5": "(b4 - b3) / (b4 + b3)",
    "pixel_type": "32R",
  }
}

In [ ]:
tool_request = { 
    "name":"San Francisco Clipped Bandmath",
    "products": product,
    "tools": [clip, bandmath],
    "delivery": {"single_archive": True, "archive_type": "zip"}
}

In [ ]:
tool_order_url = place_order(tool_request, auth)

In [ ]:
order_url = orders_url + '/396a327c-4d52-41a5-9c81-6ee91166e59e'



### View Results
Now lets review our previous order and download it

In [ ]:
requests.get(order_url, auth=auth).json()['state']

In [ ]:
r = requests.get(order_url, auth=auth)
response = r.json()
results = response['_links']['results']


In [ ]:
[r['name'] for r in results]

## Download

### Downloading each asset individually

In [ ]:
def download_results(results, overwrite=False):
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name in zip(results_urls, results_names):
        path = pathlib.Path(os.path.join('data', name))
        
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))

In [ ]:
download_results(results)

## Subscription API
#### Creating a subscription
[Here](https://developers.planet.com/docs/subscriptions/delivery/) is more info on our various cloud delivery options

In [ ]:
# set content type to json
headers = {'content-type': 'application/json'}

# set your delivery details
BUCKET_NAME = 'subscriptions_api_demo'
GCS_CREDENTIALS= #64-bit string

In [ ]:
request = {
    "name": "Tampa_Bay",
    "source": {
        "type": "catalog",
        "parameters": {
            "geometry": {
                "coordinates": [[[-82.775,27.48],[-82.365,27.48],[-82.365,28.07],[-82.775,28.07],[-82.775,27.48]]],
                "type": "Polygon"
            },
            "start_time": "2023-01-01T00:00:00Z",
            "end_time": "2023-03-31T00:00:00Z",
            "item_types": ["PSScene"],
            "asset_types": ["ortho_analytic_4b"]
        }
    },
    "delivery": { 
        "type": "google_cloud_storage",
        "parameters": {
            "bucket": BUCKET_NAME,
            "credentials": GCS_CREDENTIALS,
                
        }
    }
}

In [ ]:
def place_subscription(request, auth):
    response = requests.post(subscriptions_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response.json())
    subscriptions_id = response.json()['id']
    print(subscriptions_id)
    subscription_url = subscriptions_url + '/' + subscriptions_id
    return subscription_url